In [11]:
import os
import torch
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
from PIL import Image

In [12]:
KEYFRAME_PATH = "../../../Data/AIC2025/Keyframes"
DESCRIBE_PATH = "../../data/describe"

In [ ]:
# Thiết bị
device = "cpu"
model_id = "Qwen/Qwen2-VL-2B-Instruct"

# Load model trên CPU/Cuda nếu có
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def describe_keyframe(base_input_path, base_output_path):
    """
    Dùng mô hình để mô tả các keyframe và lưu về text
    """
    for part in os.listdir(base_input_path):
        part_path = os.path.join(base_input_path, part)
    
        # Bỏ qua ko phải dir
        if not os.path.isdir(part_path):
            continue

        # part/keyframes
        part_path = os.path.join(part_path, "keyframes")
        
        for video in os.listdir(part_path):
            video_path = os.path.join(part_path, video)
            
            for keyframe in os.listdir(video_path):
                print(f"Đang mô tả file {video}/{keyframe}")
                keyframe_path = os.path.join(video_path, keyframe)
                
                # Đọc image
                image = Image.open(keyframe_path).convert("RGB") 

                # Prompt
                messages = [
                    {
                        "role": "user",
                        "content": [
                            {"type": "image", "image": image},  # Truyền đối tượng PIL.Image
                            {"type": "text", "text": "Mô tả bức ảnh này bằng tiếng Việt thật chi tiết, súc tích."}
                        ]
                    }
                ]
                chat_template = processor.apply_chat_template(messages, add_generation_prompt=True)
                inputs = processor(
                    text=chat_template,  # Truyền template đã xử lý
                    images=[image],  # Truyền danh sách ảnh
                    return_tensors="pt"
                ).to(device)

                # Sinh mô tả
                generated = model.generate(**inputs, max_new_tokens=128)
                output = processor.batch_decode(generated, skip_special_tokens=True)[0]

                # Xuất ra txt
                # |-- Tạo part
                output_part = os.path.join(base_output_path, part)
                if not os.path.exists(output_part):
                    os.mkdir(output_part)

                # |-- Tạo output video path
                output_video = os.path.join(output_part, video)
                if not os.path.exists(output_video):
                    os.mkdir(output_video)

                # |-- Tạo txt
                output_describe = os.path.join(output_video, keyframe)
                if os.path.exists(output_describe):
                    os.remove(output_describe)
                with open(f"{output_describe}.txt", "w", encoding="utf-8") as f:
                    f.write(f"{output}")

In [15]:
describe_keyframe(KEYFRAME_PATH, DESCRIBE_PATH)

Đang mô tả file L21_V001/001.jpg
Đang mô tả file L21_V001/002.jpg
Đang mô tả file L21_V001/003.jpg
Đang mô tả file L21_V001/004.jpg
Đang mô tả file L21_V001/005.jpg
Đang mô tả file L21_V001/006.jpg
Đang mô tả file L21_V001/007.jpg
Đang mô tả file L21_V001/008.jpg
Đang mô tả file L21_V001/009.jpg
Đang mô tả file L21_V001/010.jpg
Đang mô tả file L21_V001/011.jpg
Đang mô tả file L21_V001/012.jpg
Đang mô tả file L21_V001/013.jpg
Đang mô tả file L21_V001/014.jpg
Đang mô tả file L21_V001/015.jpg
Đang mô tả file L21_V001/016.jpg
Đang mô tả file L21_V001/017.jpg
Đang mô tả file L21_V001/018.jpg
Đang mô tả file L21_V001/019.jpg
Đang mô tả file L21_V001/020.jpg
Đang mô tả file L21_V001/021.jpg
Đang mô tả file L21_V001/022.jpg
Đang mô tả file L21_V001/023.jpg
Đang mô tả file L21_V001/024.jpg
Đang mô tả file L21_V001/025.jpg
Đang mô tả file L21_V001/026.jpg
Đang mô tả file L21_V001/027.jpg
Đang mô tả file L21_V001/028.jpg
Đang mô tả file L21_V001/029.jpg
Đang mô tả file L21_V001/030.jpg
Đang mô tả

KeyboardInterrupt: 